In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np

# Financial Statment 가져오기

In [ ]:
bs = pd.read_excel('./[삼성전자]사업보고서_재무제표(2020.03.30)_ko.xls', sheet_name=1, header=6, index_col=0)

In [ ]:
bs.index

In [ ]:
new_index=list(map(str.strip, bs.index))

In [ ]:
bs.index=new_index
bs

In [ ]:
income = pd.read_excel('./[삼성전자]사업보고서_재무제표(2020.03.30)_ko.xls', sheet_name=2, header=6, index_col=0)

In [ ]:
income

In [ ]:
income.index

In [ ]:
new_index=list(map(str.strip, income.index))
income.index=new_index

In [ ]:
cost_of_debt = income.loc['금융비용','제 51 기']/(bs.loc['단기차입금','제 51 기']+bs.loc['장기차입금','제 51 기']+bs.loc['사채','제 51 기'])
cost_of_debt #49%는 말도 안되는데..? 주석을 살펴보니 금융비용아래에 외환 차익도 있다

In [ ]:
cost_of_debt = 686356/(bs.loc['단기차입금','제 51 기']+bs.loc['장기차입금','제 51 기']+bs.loc['사채','제 51 기'])
cost_of_debt

# 주가 데이터 가져와서 베타 구하기

In [ ]:
start='2015-10-07'
end='2020-10-07'

In [ ]:
yf.download('005930.KS', start=start, end=end).head()

In [ ]:
samsung = yf.download('005930.KS', start=start, end=end)['Adj Close'].to_frame()
kospi = yf.download('^KS11', start=start, end=end)['Adj Close'].to_frame()

In [ ]:
samsung.head()

In [ ]:
samsung['daily return'] = (samsung['Adj Close'] - samsung['Adj Close'].shift(1))/samsung['Adj Close'].shift(1)

In [ ]:
samsung.head()

In [ ]:
samsung['mkt daily return'] = (kospi['Adj Close'] - kospi['Adj Close'].shift(1))/kospi['Adj Close'].shift(1)

In [ ]:
samsung.tail()

In [ ]:
samsung.info()

In [ ]:
samsung=samsung.dropna()

In [ ]:
beta= np.cov(samsung['daily return'],samsung['mkt daily return'])[0][1]/np.var(samsung['mkt daily return'])

In [ ]:
beta

In [ ]:
adj_beta = 2/3 + beta/3
adj_beta

In [ ]:
risk_free = 0.01544

## CAPM

In [ ]:
cost_of_equity = risk_free + adj_beta*0.06
cost_of_equity

## WACC

In [ ]:
tax_rate = income.loc['법인세비용','제 51 기']/income.loc['법인세비용차감전순이익(손실)','제 51 기']
tax_rate

In [ ]:
debt=bs.loc['부채총계','제 51 기']
equity=bs.loc['자본총계','제 51 기']
asset=debt+equity

In [ ]:
print(debt)
print(equity)
print(debt/equity)

In [ ]:
wacc = (debt/asset)*cost_of_debt*(1-tax_rate) + (equity/asset)*cost_of_equity

In [ ]:
wacc

# ReOI model을 활용한 주가 평가하기

## 1. 현재의 NOA, NFO 값 계산

NOA =  영업관련 자산 - 영업관련 부채
NFO = 장기/단기 부채 - 현금자산

In [ ]:
bs

In [ ]:
bs.index

In [ ]:
bs.T

In [ ]:
bs2=bs.T

In [ ]:
bs2['op asset'] = bs2['매출채권']+bs2['미수금']+bs2['선급금']+bs2['선급비용']+bs2['재고자산']+bs2['기타유동자산']+bs2['유형자산']+bs2['무형자산']

In [ ]:
bs2['op asset']

In [ ]:
bs2['op lia'] = bs2['부채총계']-bs2['단기차입금']-bs2['장기차입금']-bs2['사채']

In [ ]:
bs2['op lia']

In [ ]:
bs2['NOA']=bs2['op asset']-bs2['op lia']

In [ ]:
bs2['NOA']

In [ ]:
NOA_0 = bs2.loc['제 51 기','NOA']
NOA_0

In [ ]:
bs2['NFO']=bs2['단기차입금']+bs2['장기차입금']+bs2['사채']-bs2['현금및현금성자산']

In [ ]:
bs2['NFO']

## 2. ReOI 계산

In [ ]:
income.index

In [ ]:
sales_1 = income.loc['수익(매출액)','제 51 기']*1.05
sales_1

In [ ]:
operating_profit_margin = income.loc['영업이익(손실)','제 51 기']/income.loc['수익(매출액)','제 51 기']
operating_profit_margin

In [ ]:
OI_1= sales_1*operating_profit_margin*(1-tax_rate)
OI_1

In [ ]:
ReOI_1 = OI_1 - wacc*NOA_0
ReOI_1

In [ ]:
print(sales_1/NOA_0)
NOA_turnover=1.7

In [ ]:
sales_2 = sales_1*1.07
NOA_1 = sales_2/NOA_turnover
OI_2 = sales_2*operating_profit_margin*(1-tax_rate)
ReOI_2 = OI_2-wacc*NOA_1
print(ReOI_2)

In [ ]:
sales_3 = sales_2*1.05
NOA_2 = sales_3/NOA_turnover
OI_3 = sales_3*operating_profit_margin*(1-tax_rate)
ReOI_3 = OI_3-wacc*NOA_2
print(ReOI_3)

In [ ]:
growth=0.03
sales_4 = sales_3*(growth+1)
NOA_3 = sales_4/NOA_turnover
OI_4 = sales_4*operating_profit_margin*(1-tax_rate)
ReOI_4 = OI_4-wacc*NOA_3
CV = ReOI_4/(wacc-growth)
print(CV)

In [ ]:
value_of_firm = NOA_0 + ReOI_1/(1+wacc)**1 + ReOI_2/(1+wacc)**2 + ReOI_3/(1+wacc)**3 + CV/(1+wacc)**3
value_of_debt = bs2['NFO'][0]
value_of_equity = value_of_firm - value_of_debt

In [ ]:
print(value_of_equity)

In [ ]:
number_of_shares_outstanding = 5969782550
stock_price = value_of_equity*1000000/number_of_shares_outstanding

In [ ]:
print(stock_price)

## 3. Function화

In [ ]:
NOA = [NOA_0]
sales = [income.loc['수익(매출액)','제 51 기']]
OI = []
ReOI = []
cv_ReOIandCV = []
growth = 0.03
NOA_turnover = 1.7
operating_profit_margin = income.loc['영업이익(손실)','제 51 기']/income.loc['수익(매출액)','제 51 기']
tax_rate = income.loc['법인세비용','제 51 기']/income.loc['법인세비용차감전순이익(손실)','제 51 기']
wacc = (debt/asset)*cost_of_debt*(1-tax_rate) + (equity/asset)*cost_of_equity
value_of_debt = bs2['NFO'][0]
number_of_shares_outstanding = 5969782550

In [ ]:
def stock_price(*sales_growth):
    for i,j in enumerate(sales_growth):
        temp_sales=sales[i]*(1+j)
        sales.append(temp_sales)
        
        if i!=0:
            temp_NOA=temp_sales/NOA_turnover
            NOA.append(temp_NOA)
        
        temp_OI=temp_sales*operating_profit_margin*(1-tax_rate)
        OI.append(temp_OI)
        
        temp_ReOI=temp_OI-wacc*NOA[i]
        ReOI.append(temp_ReOI)
        
        cv_ReOIandCV.append(temp_ReOI/(1+wacc)**(i+1))
        
        if i==len(sales_growth)-1:
            last_sales=temp_sales*(1+growth)
            last_OI=last_sales*operating_profit_margin*(1-tax_rate)
            last_NOA=last_sales/NOA_turnover
            last_ReOI=last_OI-wacc*last_NOA
            CV=last_ReOI/(wacc-growth)
            cv_ReOIandCV.append(CV/(1+wacc)**(i+1))
        
    print('value of firm is: ',(NOA[0]+sum(cv_ReOIandCV))*1000000)
    print('value of equity is: ', (NOA[0]+sum(cv_ReOIandCV)-value_of_debt)*1000000)
    print('expected stock price is: ', (NOA[0]+sum(cv_ReOIandCV)-value_of_debt)*1000000/number_of_shares_outstanding)

In [ ]:
NOA = [NOA_0]
sales = [income.loc['수익(매출액)','제 51 기']]
OI = []
ReOI = []
cv_ReOIandCV = []
growth = 0.03
NOA_turnover = 1.7
operating_profit_margin = income.loc['영업이익(손실)','제 51 기']/income.loc['수익(매출액)','제 51 기']
tax_rate = income.loc['법인세비용','제 51 기']/income.loc['법인세비용차감전순이익(손실)','제 51 기']
wacc = (debt/asset)*cost_of_debt*(1-tax_rate) + (equity/asset)*cost_of_equity
value_of_debt = bs2['NFO'][0]
number_of_shares_outstanding = 5969782550

stock_price(0.05,0.07,0.05)

여기서 사용된 가정:
수익 상승률, 고정된 NOA turnover, wacc 값과 wacc이 고정, leverage 고정, 고정된 operating profit margin